# Develop the crime data

This notebook will create the base grid used to train and test the crime prediction model. 
The grid will be created on a 150m x 150m grid, within the bounds of the police precincts polygon.

For each grid cell, the number of crimes will be calculated, disaggregated by the characteristics of the victim. The crimes affecting the following groups will be counted:
- All Male and Female victims
- Male victims
- Female victims

## Import the required libraries

In [1]:
import geopandas as gpd
import altair as alt
import pandas as pd
import numpy as np

## Import the police precincts shapefile
Only points within the jurisdiction of the precincts will be considered in the analysis.

In [2]:
precint_footprint = gpd.read_file('..//data//Police Precincts.geojson')

## Create the base grid for analysis

In [3]:
# Create a grid of points across the precincts to use as the center of the crime clusters
# The size of the grid is n x n, where n is the number of points in each direction
from shapely.geometry import Point, Polygon
from rtree import index
# get the bounds of the precincts
min_x, min_y, max_x, max_y = precint_footprint.total_bounds

idx = index.Index()
for i, row in precint_footprint.iterrows():
    idx.insert(i, row.geometry.bounds)

grid_size = 100  # You can adjust this value
x_points = np.linspace(min_x, max_x, grid_size)
y_points = np.linspace(min_y, max_y, grid_size)

# Round the points to 4 decimal places
x_points = np.around(x_points, 4)
y_points = np.around(y_points, 4)

grid = [Point(x, y) for x in x_points for y in y_points]

# drop the points that are not within the precincts
def is_point_inside_precincts(point, precincts_gdf, idx):
    for i in idx.intersection(point.bounds):
        if point.within(precincts_gdf.iloc[i].geometry):
            return True
    return False

filtered_grid = [point for point in grid if is_point_inside_precincts(point, precint_footprint, idx)]
filtered_grid_df = pd.DataFrame([(point.x, point.y) for point in filtered_grid], columns=['Longitude', 'Latitude'])

del grid 
del filtered_grid

## Generate the basic spatial-temporal matrix
For each point, crime probabbility will be analysed for each hour of the day and each day of the week.

In [4]:
import itertools

def point_hour_day_combinations(points, hours, days_of_week):
    for point, hour, day in itertools.product(points, hours, days_of_week):
        yield point.x, point.y, hour, day

# Convert filtered grid points to a list of Point objects
points = [Point(lon, lat) for lon, lat in filtered_grid_df[['Longitude', 'Latitude']].values]

# Define hours and days_of_week
hours = range(24)
days_of_week = range(7)

## Import the crime data

In [ ]:
with open('..//data.pickle', 'rb') as f:
    crime_data = pd.read_pickle(f)

In [ ]:
# Only keep the M and F sex values
sexes = ['M', 'F']

crime_data = crime_data.loc[crime_data['VIC_SEX'].isin(sexes)]
# Re calculate the categories for the Sex column
crime_data['VIC_SEX'] = crime_data['VIC_SEX'].cat.remove_unused_categories()

del sexes
crime_data.head(5)

In [ ]:
# filter crime_data columns to only include the columns we need
crime_data = crime_data[['CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'Longitude','Latitude', 'VIC_SEX','VIC_AGE_GROUP']]
crime_data.head(5)

# get the day of the week and the hour of the day from the date and time columns
crime_data['CMPLNT_FR_DT'] = pd.to_datetime(crime_data['CMPLNT_FR_DT'], format='%m/%d/%Y')
crime_data['CMPLNT_FR_TM'] = pd.to_datetime(crime_data['CMPLNT_FR_TM'], format='%H:%M:%S')
crime_data['day_of_week'] = crime_data['CMPLNT_FR_DT'].dt.dayofweek
crime_data['hour'] = crime_data['CMPLNT_FR_TM'].dt.hour
crime_data['year'] = crime_data['CMPLNT_FR_DT'].dt.year

In [ ]:
def nearest_grid_point(coord, grid_points):
    nearest_index = np.argmin(np.abs(coord - grid_points))
    return grid_points[nearest_index]

# Find the nearest grid point for the longitude and latitude coordinates using the defined function
crime_data['Longitude'] = crime_data['Longitude'].apply(nearest_grid_point, grid_points=x_points)
crime_data['Latitude'] = crime_data['Latitude'].apply(nearest_grid_point, grid_points=y_points)


crime_counts = (
    crime_data.groupby(['Longitude', 'Latitude', 'day_of_week', 'hour', 'year', 'VIC_SEX'])
    .size()
    .reset_index(name='count')
)

# Create a data frame with only data from 2022
crime_counts_2022 = crime_counts.loc[crime_counts['year'] == 2022]

# create a data frame all data except 2022
crime_counts = crime_counts.loc[crime_counts['year'] != 2022]

crime_datasets = [crime_counts, crime_counts_2022]

# remove the year column from the data frames
for crime_dataset in crime_datasets:
    del crime_dataset['year']

In [ ]:
for crime_counts in crime_dataset:
    crime_counts = pd.pivot_table(crime_counts, values='count', index=['Longitude', 'Latitude','hour', 'day_of_week'], columns=['VIC_SEX'], fill_value=0)
    crime_counts = crime_counts.reset_index()

## Trees

In [ ]:
# Count the trees close to each grid point
rows = None

trees = gpd.read_file('../data/2015 Street Tree Census - Tree Data.geojson', rows=rows)
trees = trees.to_crs(epsg=4326)

In [ ]:
trees['longitude'] = trees['longitude'].astype(float)
trees['latitude'] = trees['latitude'].astype(float)

trees['Longitude'] = trees['longitude'].apply(nearest_grid_point, grid_points=x_points)
trees['Latitude'] = trees['latitude'].apply(nearest_grid_point, grid_points=y_points)

In [ ]:
trees.head(5)

In [ ]:
tree_counts = (
    trees.groupby(['Longitude', 'Latitude','health'])
    .size()
    .reset_index(name='tree_count')
)

In [ ]:
tree_counts.head()

tree_counts_pivoted = pd.pivot_table(tree_counts, values='tree_count', index=['Longitude', 'Latitude'], columns=['health'], fill_value=0)
tree_counts_pivoted = tree_counts_pivoted.reset_index().rename(columns={'Good': 'good_tree_count', 'Fair': 'fair_tree_count', 'Poor': 'poor_tree_count'})

# Remove index name
tree_counts_pivoted.index.name = None


In [ ]:
all_combinations = np.array(list(point_hour_day_combinations(points, hours, days_of_week)))

In [ ]:
# Use the reindex method to index the matches directly
for crime_counts in crime_dataset:
    crime_counts.set_index(['Longitude', 'Latitude', 'hour', 'day_of_week'], inplace=True)
    crime_counts = crime_counts.reindex(
        pd.MultiIndex.from_arrays(all_combinations.T,
        names=crime_counts.index.names),
        fill_value=0
        )
    # Reset the index of the matched_data DataFrame
    crime_counts.reset_index(inplace=True)
    # Add the tree counts to the matched_df DataFrame
    crime_counts = crime_counts.merge(tree_counts_pivoted, on=['Longitude', 'Latitude'], how='left').fillna(0)
    crime_counts[['hour', 'day_of_week']] = crime_counts[['hour', 'day_of_week']].astype(int)
    #Convert the tree count columns to int type
    crime_counts[['good_tree_count', 'fair_tree_count', 'poor_tree_count']] = crime_counts[['good_tree_count', 'fair_tree_count', 'poor_tree_count']].astype(int)
    # convert the F and M columns to int type
    crime_counts[['F', 'M']] = crime_counts[['F', 'M']].astype(int)

## Final Export

In [78]:
# Save the matched_data DataFrame as a parquet file
for crime_counts in crime_dataset:
    crime_counts.to_parquet(f'../data/{str(crime_counts)}.parquet', index=False)